In [1]:
import os
import numpy as np
import h5py
from datetime import datetime
from dataclasses import dataclass
from typing import Dict, Any
from scipy import stats
import gnss_tools.signals.gps_l1ca as GPS_L1CA
import gnss_tools.time as time_utils
from gnss_tools.signals.gps_l1ca import get_GPS_L1CA_code_sequence
import gnss_tools.misc.hdf5_utils as hdf5_utils
import project_utils.sample_utils as sample_utils
import project_utils.acquisition as acquisition_utils

In [2]:
def track_GPS_L1CA_signal(
        prn: int,
        filepath: str,
        sample_params: sample_utils.SampleParameters,
        samp_rate: int,
        center_freq_hz: float,
        acq_sample_index: int,
        acq_code_phase_chips: float,
        acq_doppler_hz: float,
        **kwargs
    ) -> Dict[str, np.ndarray]:
    """
    ------------------------------------------------------------------------------------------------
    Given a PRN, acquires and tracks the corresponding GPS L1CA signal.
    
    Inputs:
        `prn` -- the PRN of the GPS satellite to acquire and track
        `source_params` -- dict containing information about the file source
        `acq_sample_index` -- the index of the sample corresponding to the acquired signal parameters
        `code_phase_acq` -- acquired signal code phase in chips
        `doppler_acq` -- acquired signal Doppler frequency in Hz
        
        `N_integration_code_periods` -- number of code periods (default 1) over which to coherently
            integrate when tracking
        `epl_chip_spacing` -- spacing of the EPL correlators in units of chips (default 0.5)
        `DLL_bandwidth` -- the bandwidth of the DLL loop filter in Hz (default 5)
        `PLL_bandwidth` -- the bandwidth of the PLL loop filter in Hz (default 20)
    
    Notes:
    
    In order to avoid data bit transitions, our tracking loop will correlate over an integer number
    of the CA code periods.  Nominally the code period is 1 ms, which at a sampling rate of 5 MHz
    comes out to 5000 samples.  However, due to Doppler expansion/compression, the actual number of
    samples will be slightly above or slightly below 5000.  We adjust the time step accordingly in
    the tracking loop, but the nominal time step is sufficient for designing our loop filter.
    """
    
    # 0. Here we set up the tracking loop
    #  Any computations we can do outside the main loop will speed up our code, improve our own lives,
    # and be better for the planet.
    
    # The `sample_loader` object will help us load samples from the binary file stream
    sample_loader = sample_utils.SampleLoader(sample_params, samp_rate, max_buffer_duration_ms=3000)
    
    # Here we define the "tracking block size" as an integer multiple of the L1CA code period.
    # Nominally, the tracking block size will be some multiple of 1 millisecond, and is equal to the
    # coherent integration time as well as the nominal tracking loop update time step.  Due to code
    # expansion / compression, the actual tracking loop update time step will be expanded or
    # compressed by a small amount.  However, this expansion / compression will not affect the design
    # of our estimators or loop filters.
    N_integration_code_periods = kwargs.get("N_integration_code_periods", 1)
    block_length_chips = GPS_L1CA.CODE_LENGTH * N_integration_code_periods
    nominal_block_duration_sec = block_length_chips / GPS_L1CA.CODE_RATE
    nominal_integration_time_sec = nominal_block_duration_sec

    # Here we define a time array that we use for sampling our reference signal
    N_block_samples = int(nominal_integration_time_sec * samp_rate)
    block_time = np.arange(N_block_samples) / samp_rate
    print(f"Nominal samples per block: {N_block_samples}")

    # Here we set how many blocks to process
    N_blocks = kwargs.get("N_blocks", None)
    if N_blocks is None:
        file_size_bytes = os.path.getsize(filepath)
        file_length_samples = file_size_bytes // sample_params.bytes_per_sample
        N_blocks = int((file_length_samples - acq_sample_index) / (samp_rate * nominal_integration_time_sec))
    
    # The EPL correlation delay spacing controls the sensitivity of the DLL to noise vs. multipath
    epl_chip_spacing = kwargs.get("epl_chip_spacing", 0.5)

    # Here we define the DLL and PLL loop filter bandwidths, with default values of 5 and 20 Hz, resp.
    DLL_bandwidth_hz = kwargs.get("DLL_bandwidth", 5)
    PLL_bandwidth_hz = kwargs.get("PLL_bandwidth", 20)

    DLL_filter_coeff = 4 * nominal_integration_time_sec * DLL_bandwidth_hz

    xi = 1 / np.sqrt(2)
    omega_n = PLL_bandwidth_hz / .53
    PLL_filter_coeffs = (
        2 * xi * omega_n * nominal_integration_time_sec 
        - 3 / 2 * omega_n**2 * nominal_integration_time_sec**2,
        omega_n**2 * nominal_integration_time_sec
    )

    # Here we preallocate our outputs
    output_keys = ["sample_index", "early", "prompt", "late",
                   "code_phase", "unfiltered_code_phase", "filtered_code_phase",
                   "carr_phase", "unfiltered_carr_phase", "filtered_carr_phase",
                   "doppler", "filtered_doppler"]
    output_dtypes = [int, complex, complex, complex,
                     float, float, float,
                     float, float, float,
                     float, float]
    outputs = {key: np.zeros(N_blocks, dtype=dtype) for key, dtype in zip(output_keys, output_dtypes)}

    # Compute the intermediate frequency
    # center_freq_hz = kwargs.get("center_freq", 1_575_420_000)
    inter_freq_hz = GPS_L1CA.CARRIER_FREQ - center_freq_hz
    print(f"Intermediate frequency: {inter_freq_hz} Hz")
    
    # Get the appropriate PRN code sequence
    code_seq = get_GPS_L1CA_code_sequence(prn)
    code_seq = 1 - 2 * code_seq  # Convert to BPSK polarity
    
    # Here we find the next sample corresponding to a start of a data bit transition.  Our
    # tracking will begin at this sample and our initial code phase will be approximately 0.
    code_rate_chips_per_sec = GPS_L1CA.CODE_RATE * (1 + acq_doppler_hz / GPS_L1CA.CARRIER_FREQ)
    # start_sample = acq_sample_index + \
    #     int(((-code_phase_acq) % (20 * GPS_L1CA.CODE_LENGTH)) * samp_rate / code_rate_chips_per_sec)
    start_sample = acq_sample_index + \
        int(((-acq_code_phase_chips) % (1 * GPS_L1CA.CODE_LENGTH)) * samp_rate / code_rate_chips_per_sec)
    print(f"Starting at sample {start_sample}")
    
    # Set tracking state variables
    code_phase_chips = acq_code_phase_chips + (start_sample - acq_sample_index) / samp_rate * code_rate_chips_per_sec
    code_rate_chips_per_sec = GPS_L1CA.CODE_RATE * (1 + acq_doppler_hz / GPS_L1CA.CARRIER_FREQ)
    carr_phase_cycles = 0
    doppler_hz = acq_doppler_hz

    print(f"Acquired code phase: {acq_code_phase_chips} chips")
    print(f"Initial code phase: {code_phase_chips} chips")
    print(f"Initial code rate: {code_rate_chips_per_sec} chips/sec")

    sample_index = start_sample
    block_index = 0

    # Open the IF sample file
    with open(filepath, "rb") as f:

        # samples = sample_loader.load_samples(f, 0, int(samp_rate * 2))
        # if samples is None:
        #     raise ValueError("No samples found in file.")
        
        # Run the tracking loop
        for block_index in range(N_blocks):

            print("\r {0: >4.1f}".format(sample_index / samp_rate), end="")
            # 1. Get the next block of samples
            sample_block = sample_loader.load_samples(f, sample_index, N_block_samples)
            # sample_block = samples[sample_index:sample_index + N_block_samples]
            # print(f"Block {block_index} at sample {sample_index}: length {len(sample_block)}")
            if sample_block is None:
                break  # end of file reached
            
            # 2. Reference Generation and Correlation
            #  For efficiency, this step is broken down into carrier wipeoff, code wipeoff, and
            # summation.  This process is equivalent to generating complete references and
            # correlating them with our IF samples.

            # 2a. Wipeoff carrier
            #  The reason we do this in a separate step is because the `cos` and `sin` operations
            # are rather expensive (but less expensive than `exp`). So if we can get away with
            # generating our carrier just once, it's worth it.  A real receiver uses custom lookups
            # for sin/cos and often does everything using integers.
            phi = 2 * np.pi * carr_phase_cycles + 2 * np.pi * (inter_freq_hz + doppler_hz) * block_time
            carrier_conj = np.cos(phi) - 1j * np.sin(phi)
            block_wo_carrier = sample_block * carrier_conj
            
            # 2b. Code wipeoff and summation
            #  Here we run a brief for-loop to obtain the 
            epl_correlations = []
            for chip_delay in [epl_chip_spacing, 0, -epl_chip_spacing]:
                chip_indices = (code_phase_chips + chip_delay + code_rate_chips_per_sec * block_time).astype(int) % GPS_L1CA.CODE_LENGTH
                code_samples = code_seq[chip_indices]
                epl_correlations.append(np.sum(block_wo_carrier * code_samples) / N_block_samples)
            early, prompt, late = epl_correlations

            # early, prompt, late = epl_correlate_bpsk(
            #         sample_block,
            #         samp_rate,
            #         code_seq,
            #         GPS_L1CA.CODE_LENGTH,
            #         code_phase_chips,
            #         code_rate_chips_per_sec,
            #         carr_phase_cycles,
            #         doppler_hz,
            #         epl_chip_spacing
            #     )
            
            # 3. Use discriminators to estimate state errors
        
            # 3a. Compute code phase error using early-minus-late discriminator
            code_phase_error = (np.abs(early) - np.abs(late)) / (np.abs(early) + np.abs(late) + 2 * np.abs(prompt))
            # code_phase_error = ### YOUR CODE HERE ###  (relevant variables `early`, `prompt`, `late`, `epl_chip_spacing`)
            
            unfiltered_code_phase = code_phase_chips + code_phase_error

            # 3b. Compute phase error (in cycles) using appropriate phase discriminator
            delta_theta = np.arctan(prompt.imag / prompt.real) ### YOUR CODE HERE ###  (I know greek letters are typically in radians, but make sure `delta_theta` is in cycles)
            
            # Note: the phase error `delta_theta` is actually equal to:
            # `carr_phase_error + integration_time + doppler_error / 2`
            # for a 2nd-order PLL, but we'll only define "unfiltered" carrier phase for our outputs.
            carr_phase_error = delta_theta
            unfiltered_carr_phase = carr_phase_cycles + carr_phase_error
            
            
            # 4. Apply loop filters to reduce noise in state error estimates
            
            # 4a. Filter code phase error to reduce noise
            #  We implement the DLL filter by updating code phase in proportion to code phase 
            # dicriminator output.  The result has the equivalent response of a 1st-order DLL filter
            filtered_code_phase_error = DLL_filter_coeff * code_phase_error
            
            filtered_code_phase = code_phase_chips + filtered_code_phase_error

            # 4b. Filter carrier phase error to reduce noise
            #  We implement the PLL filter by updating carrier phase and frequency in proportion to
            # the phase discriminator output in a way that has the equivalent response to a 2nd-order
            # PLL.
            filtered_carr_phase_error = PLL_filter_coeffs[0] * carr_phase_error
            filtered_doppler_error = PLL_filter_coeffs[1] * carr_phase_error
            
            filtered_carr_phase = carr_phase_cycles + filtered_carr_phase_error
            filtered_doppler = doppler_hz + filtered_doppler_error

            
            # 5. Save our tracking loop outputs
            outputs["sample_index"][block_index] = sample_index
            outputs["early"][block_index] = early
            outputs["prompt"][block_index] = prompt
            outputs["late"][block_index] = late
            outputs["code_phase"][block_index] = code_phase_chips
            outputs["unfiltered_code_phase"][block_index] = unfiltered_code_phase
            outputs["filtered_code_phase"][block_index] = filtered_code_phase
            outputs["carr_phase"][block_index] = carr_phase_cycles
            outputs["unfiltered_carr_phase"][block_index] = unfiltered_carr_phase
            outputs["filtered_carr_phase"][block_index] = filtered_carr_phase
            outputs["doppler"][block_index] = doppler_hz
            outputs["filtered_doppler"][block_index] = filtered_doppler

            
            # 6. Propagate state to next time epoch
            
            # As part of this step, we apply carrier-aiding by adjusting `code_rate` based on Doppler
            code_rate_chips_per_sec = GPS_L1CA.CODE_RATE * (1 + doppler_hz / GPS_L1CA.CARRIER_FREQ)
            
            # First we adjust the nominal time step to go to start of next desired chip
            target_code_phase = np.round((code_phase_chips + GPS_L1CA.CODE_LENGTH) / GPS_L1CA.CODE_LENGTH) * GPS_L1CA.CODE_LENGTH
            sample_step = int((target_code_phase - filtered_code_phase) * samp_rate / code_rate_chips_per_sec)
            time_step = sample_step / samp_rate
            
            # Then we update the states and sample index accordingly
            code_phase_chips = filtered_code_phase + code_rate_chips_per_sec * time_step
            carr_phase_cycles = filtered_carr_phase + (inter_freq_hz + doppler_hz) * time_step
            doppler_hz = filtered_doppler
            
            sample_index += sample_step

    for key in output_keys:
        outputs[key] = outputs[key][:block_index]
    outputs["prn"] = prn
    outputs["DLL_bandwidth"] = DLL_bandwidth_hz
    outputs["PLL_bandwidth"] = PLL_bandwidth_hz
    outputs["N_integration_code_periods"] = N_integration_code_periods
    outputs["integration_time"] = nominal_integration_time_sec
    outputs["time"] = outputs["sample_index"] / samp_rate
    
    return outputs

In [3]:
# Choose IF data file and appropriate data parameters

# This should be the path to raw IF data file that you download from the class site
user_data_dir = "../local-data/"

sample_filepath = os.path.join(user_data_dir, "raw-collects/local-data/raw-collects/CO_A2_20220513_173218_000211_G1_B1_USRP2.sc4")
file_start_time_dt = datetime(2021, 6, 11, 12, 10)

sample_file_id = os.path.basename(sample_filepath)[:-4]

# The file contains complex 4-bit samples, where the first byte is the real component
# The sampling rate is 5 MHz and the front-end center frequency is at GPS L1
sample_params = sample_utils.SampleParameters(
    4, True, True, True, False
)
samp_rate = 25_000_000
center_freq_hz = 1_570_000_000

file_start_time_gpst = time_utils.convert_datetime_to_gps_seconds(file_start_time_dt)

In [ ]:
# Load acq. results
acq_results_version_id = "0001"
acq_results_dir = os.path.join(user_data_dir, "acq-outputs")
acq_results_filepath = os.path.join(acq_results_dir, f"{sample_file_id}_{acq_results_version_id}.h5")
with h5py.File(acq_results_filepath, "r") as f:
    acq_results_dict = hdf5_utils.read_hdf5_into_dict(f)

acq_N_ncoh = acq_results_dict["N_ncoh"]
acq_N_coh = acq_results_dict["N_coh"]
acq_results = acq_results_dict["results"]

noise_dist = stats.chi2(df=2 * acq_N_ncoh)
acq_pfd_total = 1e-3
acq_pfd_test = 1 - np.exp(np.log(1 - acq_pfd_total) / acq_N_coh)
acq_threshold = noise_dist.ppf(1 - acq_pfd_test) / acq_N_ncoh
print(f"Acquisition threshold: {10 * np.log10(acq_threshold)} dB")

sat_ids = []
for sat_id, result in acq_results.items():
    if result["snr"] > acq_threshold:
        sat_ids.append(sat_id)
    print(
        f"{sat_id}: {10 * np.log10(result['snr']):7.3f} dB, {result['code_phase']:7.3f} chips, {result['doppler']:7.3f} Hz"
    )

Acquisition threshold: 13.028139050310603 dB
G01:  12.649 dB, 736.969 chips, 4200.000 Hz
G02:  25.418 dB, 548.123 chips, -1200.000 Hz
G03:  11.881 dB, 417.998 chips, 3200.000 Hz
G04:  12.443 dB, 1013.179 chips, -2800.000 Hz
G05:  15.952 dB, 277.028 chips, 2400.000 Hz
G06:  24.440 dB, 442.141 chips, -1800.000 Hz
G07:  12.471 dB, 553.238 chips, -2800.000 Hz
G08:  12.695 dB, 914.153 chips, 4200.000 Hz
G09:  18.007 dB, 722.033 chips, -1600.000 Hz
G10:  12.391 dB,  37.237 chips, 2200.000 Hz
G11:  12.067 dB, 864.026 chips, -3800.000 Hz
G12:  12.805 dB, 754.360 chips, 1200.000 Hz
G13:  17.938 dB, 790.370 chips, 2400.000 Hz
G14:  13.932 dB, 472.012 chips, -1000.000 Hz
G15:  11.979 dB, 341.068 chips, -3800.000 Hz
G16:  11.912 dB, 128.080 chips, 1200.000 Hz
G17:  22.340 dB, 683.978 chips, 2200.000 Hz
G18:  13.021 dB, 185.163 chips, 200.000 Hz
G19:  25.898 dB, 494.109 chips, 2000.000 Hz
G20:  12.216 dB, 750.064 chips, 4200.000 Hz
G21:  12.191 dB, 813.080 chips, 200.000 Hz
G22:  12.180 dB, 759.271

In [5]:
tracking_kwargs = {
    "N_integration_code_periods": 1,
    "DLL_bandwidth": 5,
    "PLL_bandwidth": 20,
    "epl_chip_spacing": 0.5,
    "use_adaptive_code_period_tracking": True,
}
version_id = "0001"

# Write outputs to file
OVERWRITE_TRACKING_OUTPUTS = True
tracking_output_dir = os.path.join(
    user_data_dir,
    f"asen-6092/tracking-outputs/{sample_file_id}/{version_id}/"
)
os.makedirs(tracking_output_dir, exist_ok=True)

tracking_results = {}
for sat_id in sat_ids:
    print(f"Tracking {sat_id}...")
    output_filename = "tracking-{0}_N-int-{1:02}_DLL-BW-{2:02}_PLL-BW-{3:02}.h5".format(
        sat_id,
        tracking_kwargs["N_integration_code_periods"],
        tracking_kwargs["DLL_bandwidth"],
        tracking_kwargs["PLL_bandwidth"],
    )
    output_filepath = os.path.join(tracking_output_dir, output_filename)
    if not os.path.exists(output_filepath) or OVERWRITE_TRACKING_OUTPUTS:
        prn = int(sat_id[1:])
        c_acq, f_acq, n_acq = acquisition_utils.acquire_GPS_L1CA_signal(
            sample_filepath,
            samp_rate,
            center_freq_hz,
            sample_params,
            prn,
            0
        )
        acq_code_phase_chips = n_acq["code_phase"]
        acq_doppler_hz = f_acq["doppler"]
        # sat_acq_result = acq_results[sat_id]
        # acq_code_phase_chips = sat_acq_result["code_phase"]
        # acq_doppler_hz = sat_acq_result["doppler"]
        tracking = track_GPS_L1CA_signal(
            prn,
            sample_filepath,
            sample_params,
            samp_rate,
            center_freq_hz,
            0,
            acq_code_phase_chips,
            acq_doppler_hz,
            **tracking_kwargs
        )
        print("")
        tracking_results[sat_id] = {}
        tracking_results[sat_id]["c_acq"] = c_acq
        tracking_results[sat_id]["f_acq"] = f_acq
        tracking_results[sat_id]["n_acq"] = n_acq
        tracking_results[sat_id]["tracking"] = tracking

        with h5py.File(output_filepath, "w") as f:
            hdf5_utils.write_dict_to_hdf5(tracking_results[sat_id], f)
    else:
        with h5py.File(output_filepath, "r") as f:
            tracking_results[sat_id] = hdf5_utils.read_hdf5_into_dict(f)

Tracking G02...
Code Phase: 548.123 chips 	Doppler Freq: -1055.035 	 C/N0: 51.519
Phi0: -3.103 rad 	Doppler Freq: -1103.043 	 Dopp. Delta: -48.008
Data Bit Phase: 17.000
Nominal samples per block: 5000
Intermediate frequency: 0 Hz
Starting at sample 2321
Acquired code phase: 16916.1234 chips
Initial code phase: 17390.999667511325 chips
Initial code rate: 1022999.2837383127 chips/sec
 60.0
Tracking G05...
Code Phase: 107.210 chips 	Doppler Freq: -3855.035 	 C/N0: 40.676
Phi0: 0.266 rad 	Doppler Freq: -3879.634 	 Dopp. Delta: -24.598
Data Bit Phase: 0.000
Nominal samples per block: 5000
Intermediate frequency: 0 Hz
Starting at sample 4476
Acquired code phase: -915.7896 chips
Initial code phase: -0.0022552260501242927 chips
Initial code rate: 1022997.4807573167 chips/sec
 60.0
Tracking G06...
Code Phase: 442.141 chips 	Doppler Freq: -1855.035 	 C/N0: 52.859
Phi0: -0.542 rad 	Doppler Freq: -1891.961 	 Dopp. Delta: -36.926
Data Bit Phase: 17.000
Nominal samples per block: 5000
Intermediate 